# Semana 18: Análisis de negocio 💼

## Economía unitaria ⚖️
La economía unitaria evalúa la rentabilidad por unidad: puede ser por producto, por cliente o por transacción. Este análisis permite tomar decisiones estratégicas como escalar campañas, ajustar precios o modificar el modelo de negocio.

### Economía unitaria por ventas 💹
Mide cuánto ganas por cada venta, restando los costos asociados. Es útil para entender si vender más unidades realmente significa mayor rentabilidad.

In [1]:
import pandas as pd

productos = pd.DataFrame({
    'producto': ['A', 'B', 'C'],
    'precio_venta': [100, 150, 200],
    'costo_unitario': [60, 80, 120]
})

productos['ganancia_unidad'] = productos['precio_venta'] - productos['costo_unitario']

productos

,producto,precio_venta,costo_unitario,ganancia_unidad
0,A,100,60,40
1,B,150,80,70
2,C,200,120,80


### Economía unitaria por ventas: crear un modelo 💻
Crear un modelo de economía unitaria te permite simular cambios: ¿qué pasa si el costo sube? ¿y si suben las ventas?

In [2]:
def modelo_venta(precio, costo, unidades):
    ingreso = precio * unidades
    costo_total = costo * unidades
    ganancia = ingreso - costo_total
    return {
        'Ingreso': ingreso,
        'Costo': costo_total,
        'Ganancia': ganancia,
        'Ganancia por unidad': precio - costo
    }

print(modelo_venta(precio=90, costo=50, unidades=1000))

{'Ingreso': 90000, 'Costo': 50000, 'Ganancia': 40000, 'Ganancia por unidad': 40}


### Economía unitaria por cliente: LTV y CAC 🪪
**CAC** (Costo de adquisición del cliente): cuánto cuesta atraer un nuevo cliente.

**LTV** (Valor del cliente a lo largo del tiempo): cuánto dinero genera un cliente promedio.

In [3]:
clientes = pd.DataFrame({
    'cliente_id': range(1, 6),
    'ingresos_totales': [300, 500, 200, 400, 150],
    'num_meses_activo': [3, 5, 2, 4, 1],
    'costo_adquisicion': [50, 70, 40, 60, 30]
})

clientes['LTV'] = clientes['ingresos_totales']
clientes['CAC'] = clientes['costo_adquisicion']
clientes['LTV_CAC_ratio'] = clientes['LTV'] / clientes['CAC']

clientes[['cliente_id', 'LTV', 'CAC', 'LTV_CAC_ratio']]

,cliente_id,LTV,CAC,LTV_CAC_ratio
0,1,300,50,6.000000
1,2,500,70,7.142857
2,3,200,40,5.000000
3,4,400,60,6.666667
4,5,150,30,5.000000


### Economía unitaria por cliente para una tienda online 🌐
La tienda online puede calcular LTV usando el ticket promedio y la frecuencia de compra, y compararlo con el CAC para evaluar rentabilidad.

In [4]:
clientes_online = pd.DataFrame({
    'cliente_id': [1, 2, 3, 4],
    'total_compras': [3, 7, 2, 5],
    'valor_promedio_compra': [50, 80, 30, 60],
    'costo_logistica': [20, 35, 10, 25],
    'CAC': [30, 40, 20, 35]
})

clientes_online['ingreso_total'] = clientes_online['total_compras'] * clientes_online['valor_promedio_compra']
clientes_online['costo_total'] = clientes_online['costo_logistica'] + clientes_online['CAC']
clientes_online['ganancia_neta'] = clientes_online['ingreso_total'] - clientes_online['costo_total']

clientes_online

,cliente_id,total_compras,valor_promedio_compra,costo_logistica,CAC,ingreso_total,costo_total,ganancia_neta
0,1,3,50,20,30,150,50,100
1,2,7,80,35,40,560,75,485
2,3,2,30,10,20,60,30,30
3,4,5,60,25,35,300,60,240


## Métricas de usuario 📊
Más allá de las visitas o clics, medimos cuánto nos cuesta adquirirlo, cuánto nos deja y cómo evoluciona a lo largo del tiempo. Esto se conecta con el modelo de negocio y decisiones de crecimiento.

### Evaluación de la actividad del usuario 🧾

In [5]:
import numpy as np

eventos = pd.DataFrame({
    'usuario_id': np.random.randint(1, 11, 100),
    'evento': np.random.choice(['visita', 'click', 'compra'], 100),
    'timestamp': pd.date_range(start='2024-06-01', periods=100, freq='H')
})

actividad = eventos.groupby('usuario_id')['evento'].count()

actividad

/tmp/ipykernel_8170/92739961.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'timestamp': pd.date_range(start='2024-06-01', periods=100, freq='H')


usuario_id
1     14
2      8
3      8
4     13
5     11
6     10
7      7
8     11
9      6
10    12
Name: evento, dtype: int64

### Sesiones de usuario 🖥️
Una sesión es una secuencia de eventos consecutivos sin largos períodos de inactividad (ej. 30 minutos).

In [6]:
eventos['timestamp'] = pd.to_datetime(eventos['timestamp'])
eventos = eventos.sort_values(['usuario_id', 'timestamp'])
eventos['diferencia'] = eventos.groupby('usuario_id')['timestamp'].diff()
eventos['nueva_sesion'] = eventos['diferencia'] > pd.Timedelta(minutes=30)
eventos['sesion_id'] = eventos.groupby('usuario_id')['nueva_sesion'].cumsum().fillna(0)

eventos[['usuario_id', 'timestamp', 'evento', 'sesion_id']].head(10)

,usuario_id,timestamp,evento,sesion_id
2,1,2024-06-01 02:00:00,visita,0
22,1,2024-06-01 22:00:00,compra,1
23,1,2024-06-01 23:00:00,click,2
24,1,2024-06-02 00:00:00,visita,3
35,1,2024-06-02 11:00:00,compra,4
41,1,2024-06-02 17:00:00,click,5
53,1,2024-06-03 05:00:00,compra,6
57,1,2024-06-03 09:00:00,visita,7
60,1,2024-06-03 12:00:00,compra,8
63,1,2024-06-03 15:00:00,visita,9


### Frameworks de métricas 🗃️
Los más conocidos son:
- **AARRR** (Acquisition, Activation, Retention, Referral, Revenue)
- **HEART** (Happiness, Engagement, Adoption, Retention, Task success)

In [7]:
usuarios = pd.DataFrame({
    'usuario_id': [1, 2, 3, 4, 5],
    'visitas': [3, 10, 5, 7, 2],
    'activaciones': [1, 3, 2, 0, 1],
    'retenciones': [1, 2, 0, 1, 1],
    'compras': [0, 2, 1, 0, 0]
})

usuarios['fase'] = usuarios.apply(
    lambda row: 'Revenue' if row['compras'] > 0 else
                'Retention' if row['retenciones'] > 0 else
                'Activation' if row['activaciones'] > 0 else
                'Acquisition',
    axis=1
)

usuarios

,usuario_id,visitas,activaciones,retenciones,compras,fase
0,1,3,1,1,0,Retention
1,2,10,3,2,2,Revenue
2,3,5,2,0,1,Revenue
3,4,7,0,1,0,Retention
4,5,2,1,1,0,Retention


### Detección de anomalías 🔍

In [8]:
# Simulamos valores de tráfico
trafico = pd.DataFrame({
    'fecha': pd.date_range('2024-06-01', periods=30),
    'visitas': np.random.normal(1000, 100, 30).astype(int)
})

# Introducir una anomalía
trafico.loc[10, 'visitas'] = 3000

# Detección simple con z-score
from scipy.stats import zscore
trafico['zscore'] = zscore(trafico['visitas'])
anomalías = trafico[np.abs(trafico['zscore']) > 2]

anomalías

,fecha,visitas,zscore
10,2024-06-11,3000,5.185893


### Análisis web 🌐
Analizar visitas, duración de sesión, conversión, tráfico por canal, etc.

In [9]:
trafico_web = pd.DataFrame({
    'canal': ['orgánico', 'pago', 'referencia', 'directo'],
    'visitas': [3000, 2000, 500, 1500],
    'conversiones': [150, 100, 25, 75]
})

trafico_web['CR'] = trafico_web['conversiones'] / trafico_web['visitas']

trafico_web

,canal,visitas,conversiones,CR
0,orgánico,3000,150,0.05
1,pago,2000,100,0.05
2,referencia,500,25,0.05
3,directo,1500,75,0.05


### Trabajar con datos sin procesar 📚

In [10]:
import json

raw_data = '[{"user":1,"action":"visit","timestamp":"2024-06-01T10:00:00"}, {"user":1,"action":"buy","timestamp":"2024-06-01T10:10:00"}]'
parsed = pd.json_normalize(json.loads(raw_data))

print(parsed)

   user action            timestamp
0     1  visit  2024-06-01T10:00:00
1     1    buy  2024-06-01T10:10:00
